In [1]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()

if "GPU" not in device_name:
    print("GPU device not found")
else:
    print(f'Found GPU at: {device_name}')

# Alternative way to check
print("GPU", "available (YESS!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")


2025-05-18 17:19:23.268652: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747588763.467436      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747588763.523232      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found GPU at: /device:GPU:0
GPU available (YESS!!!!)


I0000 00:00:1747588776.096127      35 gpu_device.cc:2022] Created device /device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [2]:
!pip install tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 13.5 MB/s eta 0:00:0000:01
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.2
    Uninstalling typeguard-4.4.2:
      Successfully uninstalled typeguard-4.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.16.1 requires typeguard<5,>=3, but you have typeguard 2.13.3 which is incompatible.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [3]:
!pip install transformers

# **PART 2**

In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cufft-

In [5]:
import logging
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import load_dataset
from torch.optim import AdamW
import pandas as pd

# Suppress transformer warnings
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

# Load tokenizer and dataset
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
from datasets import load_dataset

# Load the Tamil NER dataset
dataset = load_dataset("google/xtreme", "PAN-X.te")
train_data = dataset['train']
validation_data = dataset['validation']
test_data = dataset['test']

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/131k [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/65.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/67.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [6]:
# Preprocessing the dataset to align NER tags with tokens
label_list = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]

def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    previous_word_id = None
    for word_id in word_ids:
        if word_id is None:
            new_labels.append(-100)  # Ignore these tokens in the loss
        elif word_id != previous_word_id:
            new_labels.append(labels[word_id])  # Only label first token of the word
        else:
            # Continue with I- tags for inner tokens, otherwise ignore
            new_labels.append(labels[word_id] if label_list[labels[word_id]].startswith("I-") else -100)
        previous_word_id = word_id
    return new_labels


def preprocess_data(batch):
    tokenized_inputs = tokenizer(batch["tokens"], truncation=True, is_split_into_words=True, padding="max_length", max_length=128)
    all_word_ids = [tokenized_inputs.word_ids(i) for i in range(len(batch["tokens"]))]

    # Align labels with word ids for each example in the batch
    all_aligned_labels = [
        align_labels_with_tokens(labels, word_ids) for labels, word_ids in zip(batch["ner_tags"], all_word_ids)
    ]

    # Add aligned labels to tokenized inputs
    tokenized_inputs["labels"] = all_aligned_labels
    return tokenized_inputs


train_dataset = train_data.map(preprocess_data, batched=True)
validation_dataset = validation_data.map(preprocess_data, batched=True)
test_dataset = test_data.map(preprocess_data, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:
# Load model
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

# Load the model with more capacity for multilingual tasks
model = AutoModelForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels=dataset['train'].features['ner_tags'].feature.num_classes)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [8]:
!pip install seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=203a0290d24435f28327e7780df1f26e9fddaba2641517a0e3454990d61475c8
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [9]:
from seqeval.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score
import numpy as np

# Define compute_metrics function
def compute_metrics(pred):
    label_ids = pred.label_ids
    predictions = pred.predictions.argmax(-1)
    
    # Remove ignored index (special tokens)
    true_labels = [[label_list[label_id] for (label_id, pred_id) in zip(label_ids[i], predictions[i]) if label_id != -100]
                   for i in range(len(label_ids))]
    true_predictions = [[label_list[pred_id] for (label_id, pred_id) in zip(label_ids[i], predictions[i]) if label_id != -100]
                        for i in range(len(predictions))]
    
    # Calculate metrics using seqeval
    precision = precision_score(true_labels, true_predictions)
    recall = recall_score(true_labels, true_predictions)
    f1 = f1_score(true_labels, true_predictions)
    accuracy = accuracy_score(true_labels, true_predictions)
    
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [10]:
# A-LLRD Optimizer (fixed for mBERT and checkpointing)
def get_optimizer(model, base_lr=2e-5, min_decay=0.85, max_decay=0.95):
    def compute_gradient_norms(model):
        norms = []
        for i, layer in enumerate(model.bert.encoder.layer):
            norm = 0
            for param in layer.parameters():
                if param.grad is not None:
                    norm += param.grad.norm().item() ** 2
            norms.append(np.sqrt(norm) if norm > 0 else 1e-8)
        return norms

    class ALLRDOptimizer(AdamW):
        def __init__(self, params, model, base_lr, min_decay, max_decay, betas=(0.9, 0.999), eps=1e-8, amsgrad=False, weight_decay=0.01, fused=False, capturable=False, differentiable=False, maximize=False, foreach=None):
            defaults = dict(
                lr=base_lr,
                betas=betas,
                eps=eps,
                amsgrad=amsgrad,
                weight_decay=weight_decay,
                fused=fused,
                capturable=capturable,
                differentiable=differentiable,
                maximize=maximize,
                foreach=foreach
            )
            super().__init__(params, **defaults)
            self.model = model
            self.base_lr = base_lr
            self.min_decay = min_decay
            self.max_decay = max_decay
            self.step_count = 0

        def step(self, closure=None):
            self.step_count += 1
            norms = compute_gradient_norms(self.model)
            norms = [min(max(norm, 1e-8), 5.0) for norm in norms]
            norms = torch.tensor(norms, device=next(self.model.parameters()).device)
            decay_factors = torch.softmax(-norms, dim=0).cpu().numpy()
            decay_factors = self.min_decay + (self.max_decay - self.min_decay) * decay_factors / decay_factors.max()
            if self.step_count % 100 == 0:
                print(f"Step {self.step_count}: Decay factors: {decay_factors.tolist()}")
            # Update learning rates of existing param_groups instead of rebuilding
            for i, param_group in enumerate(self.param_groups[:-1]):  # Exclude classifier group
                param_group['lr'] = self.base_lr * decay_factors[i]
            self.param_groups[-1]['lr'] = self.base_lr  # Classifier LR remains fixed
            super().step(closure)

    # Initialize parameter groups (only once)
    optimizer_grouped_parameters = [
        {
            "params": layer.parameters(),
            "lr": base_lr,
            "betas": (0.9, 0.999),
            "eps": 1e-8,
            "amsgrad": False,
            "weight_decay": 0.01,
            "fused": False,
            "capturable": False,
            "differentiable": False,
            "maximize": False,
            "foreach": None
        } for layer in model.bert.encoder.layer
    ] + [{
        "params": model.classifier.parameters(),
        "lr": base_lr,
        "betas": (0.9, 0.999),
        "eps": 1e-8,
        "amsgrad": False,
        "weight_decay": 0.01,
        "fused": False,
        "capturable": False,
        "differentiable": False,
        "maximize": False,
        "foreach": None
    }]
    return ALLRDOptimizer(optimizer_grouped_parameters, model, base_lr, min_decay, max_decay)

In [11]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Updated to eval_strategy as requested
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.05,
    report_to="none",
    logging_steps=100,
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    save_total_limit=1
)

# Initialize trainer (without early stopping)
data_collator = DataCollatorForTokenClassification(tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(get_optimizer(model, base_lr=2e-5, min_decay=0.85, max_decay=0.95), None)
)

# Train the model
trainer.train()
metrics = trainer.evaluate(test_dataset)
metrics_df = pd.DataFrame([metrics])
print(metrics_df)

/tmp/ipykernel_35/772242240.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.679800,0.300865,0.906367,0.636710,0.696087,0.665076
2,0.260200,0.259872,0.917146,0.684598,0.773522,0.726349
3,0.202900,0.361369,0.912305,0.663454,0.745212,0.701961
4,0.103600,0.334519,0.927469,0.726261,0.815154,0.768144
5,0.047700,0.327434,0.923906,0.726227,0.788510,0.756088
6,0.050300,0.426629,0.926007,0.716495,0.810158,0.760453
7,0.042400,0.418224,0.926373,0.729730,0.809326,0.767469
8,0.022600,0.454523,0.928108,0.740045,0.820150,0.778041
9,0.041200,0.471229,0.926647,0.741185,0.822648,0.779795
10,0.019100,0.486496,0.927560,0.743012,0.796836,0.768984


Step 100: Decay factors: [0.9410034754440825, 0.9400025097641224, 0.9389889503071149, 0.9371083349638716, 0.9350208963869351, 0.9352453631526736, 0.9400797080510572, 0.9409716274171169, 0.9416081653277918, 0.9442396041999119, 0.95, 0.949913059239195]
Step 200: Decay factors: [0.935936082128663, 0.9310472535674814, 0.9277132282330565, 0.9256809126638857, 0.927206899526274, 0.9315986872106244, 0.9392426904501083, 0.9390099780217019, 0.9440703489998465, 0.9471816799069237, 0.95, 0.9484174723304368]
Step 300: Decay factors: [0.9336823800297238, 0.9310328420715841, 0.9303731370743558, 0.9265245181820335, 0.9243194686904991, 0.9209506958382628, 0.9299726354055721, 0.9337577147678815, 0.9352577295206642, 0.9424077369145216, 0.9483813926912787, 0.95]
Step 400: Decay factors: [0.9406217979723446, 0.9391142938100054, 0.9358554799384683, 0.9283255690341508, 0.9274978259892295, 0.9274329373387705, 0.9356460598893401, 0.9390720037205541, 0.9428881126130492, 0.9465711318992664, 0.95, 0.9484370237028

   eval_loss  eval_accuracy  eval_precision  eval_recall  eval_f1  \
0    0.47709       0.924328        0.721249     0.808711  0.76248   

   eval_runtime  eval_samples_per_second  eval_steps_per_second  epoch  
0        4.2903                  233.083                 29.135   10.0  


In [12]:
# Inspect the log history to understand the structure of logs
for log in trainer.state.log_history:
    print(log)


{'loss': 0.6798, 'grad_norm': 4.169421195983887, 'learning_rate': 1.8416e-05, 'epoch': 0.8, 'step': 100}
{'eval_loss': 0.30086463689804077, 'eval_accuracy': 0.9063670411985019, 'eval_precision': 0.6367098248286367, 'eval_recall': 0.6960865945045795, 'eval_f1': 0.6650755767700876, 'eval_runtime': 4.2967, 'eval_samples_per_second': 232.737, 'eval_steps_per_second': 29.092, 'epoch': 1.0, 'step': 125}
{'loss': 0.2602, 'grad_norm': 11.677875518798828, 'learning_rate': 1.6816e-05, 'epoch': 1.6, 'step': 200}
{'eval_loss': 0.25987178087234497, 'eval_accuracy': 0.9171462501141865, 'eval_precision': 0.6845983787767134, 'eval_recall': 0.7735220649458784, 'eval_f1': 0.7263487099296326, 'eval_runtime': 4.296, 'eval_samples_per_second': 232.777, 'eval_steps_per_second': 29.097, 'epoch': 2.0, 'step': 250}
{'loss': 0.2029, 'grad_norm': 5.723728179931641, 'learning_rate': 1.5216000000000001e-05, 'epoch': 2.4, 'step': 300}
{'eval_loss': 0.36136895418167114, 'eval_accuracy': 0.9123047410249383, 'eval_pre

In [13]:
# Log validation metrics per epoch
print("\nValidation Metrics per Epoch:")
validation_metrics = []
for log in trainer.state.log_history:
    if "eval_f1" in log:
        epoch_metrics = {
            "epoch": log.get("epoch", None),
            "eval_loss": log.get("eval_loss", None),
            "eval_accuracy": log.get("eval_accuracy", None),
            "eval_precision": log.get("eval_precision", None),
            "eval_recall": log.get("eval_recall", None),
            "eval_f1": log.get("eval_f1", None)
        }
        validation_metrics.append(epoch_metrics)
validation_df = pd.DataFrame(validation_metrics)
print(validation_df)

# Evaluate on test dataset (overall metrics)
print("\nTest Metrics:")
test_metrics = trainer.evaluate(test_dataset)
test_metrics_summary = {
    "test_accuracy": test_metrics["eval_accuracy"],
    "test_precision": test_metrics["eval_precision"],
    "test_recall": test_metrics["eval_recall"],
    "test_f1": test_metrics["eval_f1"]
}
test_metrics_df = pd.DataFrame([test_metrics_summary])
print(test_metrics_df)

# Compute per-class metrics manually
print("\nPer-Class Test Metrics:")
model.eval()
all_true_labels = []
all_pred_labels = []

for batch in trainer.get_test_dataloader(test_dataset):
    batch = {k: v.to(model.device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    predictions = outputs.logits.argmax(dim=-1).cpu().numpy()
    labels = batch["labels"].cpu().numpy()
    for i in range(len(labels)):
        true_labels = [label_list[l] for l, p in zip(labels[i], predictions[i]) if l != -100]
        pred_labels = [label_list[p] for l, p in zip(labels[i], predictions[i]) if l != -100]
        all_true_labels.extend(true_labels)
        all_pred_labels.extend(pred_labels)

class_report = classification_report([all_true_labels], [all_pred_labels], output_dict=True)
per_class_data = []
for label, metrics in class_report.items():
    if label in label_list:
        per_class_data.append({
            "label": label,
            "precision": metrics["precision"],
            "recall": metrics["recall"],
            "f1-score": metrics["f1-score"],
            "support": metrics["support"]
        })
per_class_df = pd.DataFrame(per_class_data)
print(per_class_df)


Validation Metrics per Epoch:
    epoch  eval_loss  eval_accuracy  eval_precision  eval_recall   eval_f1
0     1.0   0.300865       0.906367        0.636710     0.696087  0.665076
1     2.0   0.259872       0.917146        0.684598     0.773522  0.726349
2     3.0   0.361369       0.912305        0.663454     0.745212  0.701961
3     4.0   0.334519       0.927469        0.726261     0.815154  0.768144
4     5.0   0.327434       0.923906        0.726227     0.788510  0.756088
5     6.0   0.426629       0.926007        0.716495     0.810158  0.760453
6     7.0   0.418224       0.926373        0.729730     0.809326  0.767469
7     8.0   0.454523       0.928108        0.740045     0.820150  0.778041
8     9.0   0.471229       0.926647        0.741185     0.822648  0.779795
9    10.0   0.486496       0.927560        0.743012     0.796836  0.768984
10   10.0   0.477090       0.924328        0.721249     0.808711  0.762480

Test Metrics:
   test_accuracy  test_precision  test_recall  test_f1

In [16]:
# Test model on example sentences
def test_model(sentences):
    model.eval()
    for sentence in sentences:
        tokenized_input = tokenizer(sentence, truncation=True, padding="max_length", max_length=128, return_tensors="pt")
        input_ids = tokenized_input["input_ids"].to(model.device)
        attention_mask = tokenized_input["attention_mask"].to(model.device)
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
        predictions = np.argmax(outputs.logits.cpu().numpy(), axis=2)
        tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu().numpy())
        predicted_labels = [label_list[p] for p in predictions[0]]
        print(f"\nTesting sentence: {sentence}")
        for token, label in zip(tokens, predicted_labels):
            if token not in tokenizer.special_tokens_map.values() and token.strip():
                print(f"Word: '{token}', Predicted Label: {label}")

# Telugu test sentences
telugu_test_sentences = [
    "నా పేరు అరుణ్ కుమార్. నేను చెన్నై నుండి వస్తున్నాను."  #arun kumar
    "నా పేరు రాహుల్ గాంధీ, నేను ఆంధ్రప్రదేశ్ నుండి వచ్చాను.",
    "ఇది ఒక కొత్త టెక్ కంపెనీ, టాటా టెక్నాలజీస్, ప్రారంభం.",
    "తెలుగు భాష ఆంధ్రప్రదేశ్ యొక్క అధికారిక భాష.",
    "ఈ మొక్క భారతదేశం, చైనా, జపాన్, పాపువా న్యూ గినియా, మైక్రోనేషియా వంటి దేశాలలో కనిపిస్తుంది.",
    "పవన్ కళ్యాణ్, చిరంజీవి, నాగార్జున, ప్రభాస్, అల్లు అర్జున్ తెలుగు సినిమా యొక్క ప్రసిద్ధ నటులు."
]

test_model(telugu_test_sentences)


Testing sentence: నా పేరు అరుణ్ కుమార్. నేను చెన్నై నుండి వస్తున్నాను.నా పేరు రాహుల్ గాంధీ, నేను ఆంధ్రప్రదేశ్ నుండి వచ్చాను.
Word: 'న', Predicted Label: O
Word: '##ా', Predicted Label: O
Word: 'పేరు', Predicted Label: O
Word: 'అ', Predicted Label: B-PER
Word: '##రు', Predicted Label: I-PER
Word: '##ణ', Predicted Label: I-PER
Word: '##్', Predicted Label: I-PER
Word: 'కు', Predicted Label: I-PER
Word: '##మా', Predicted Label: I-PER
Word: '##ర్', Predicted Label: I-PER
Word: '.', Predicted Label: O
Word: 'న', Predicted Label: O
Word: '##ే', Predicted Label: O
Word: '##ను', Predicted Label: O
Word: 'చ', Predicted Label: B-LOC
Word: '##ెన్', Predicted Label: B-LOC
Word: '##న', Predicted Label: I-LOC
Word: '##ై', Predicted Label: I-LOC
Word: 'నుండి', Predicted Label: O
Word: 'వ', Predicted Label: O
Word: '##స్', Predicted Label: O
Word: '##తున్న', Predicted Label: O
Word: '##ాను', Predicted Label: O
Word: '.', Predicted Label: O
Word: 'న', Predicted Label: O
Word: '##ా', Predicted Label: O